In [1]:
from alpha_vantage.timeseries import TimeSeries
import os
import pandas as pd
import requests
import matplotlib.pyplot as plt

In [ ]:
API_KEY = "YBKMA5L9K9I3GE0E"


In [6]:
import requests
import pandas as pd

API_KEY = "WGWW2PTZ1ESFJ2OO"
symbol = "SPY"
interval = "15 min"

def get_intraday_data(symbol, interval="5min"):
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval={interval}&apikey={API_KEY}&outputsize=compact"
    response = requests.get(url).json()
    df = pd.DataFrame.from_dict(response[f"Time Series ({interval})"], orient="index")
    df.columns = ["Open", "High", "Low", "Close", "Volume"]
    df = df.astype(float)
    return df

# Cambia "5min" por "1min", "15min", "30min", "1h"
df_spy = get_intraday_data("SPY", "15min")
print(df_spy.head())


KeyError: 'Time Series (15min)'

In [18]:
df_spy.reset_index(inplace=True)
df_spy.rename(columns={"index": "Time"}, inplace=True)
df_spy.head()

,Time,Open,High,Low,Close,Volume
0,2025-02-27 20:00:00,585.050,585.05,585.05,585.0500,1625705.0
1,2025-02-27 19:45:00,585.190,585.27,584.88,584.8800,28517.0
2,2025-02-27 19:30:00,585.560,585.73,585.11,585.2299,16223.0
3,2025-02-27 19:15:00,585.960,586.04,585.39,585.5600,18956.0
4,2025-02-27 19:00:00,586.045,586.26,585.96,585.9600,16332.0


In [26]:
import plotly
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

pd.options.plotting.backend = "plotly"

# Asegúrate de que `df_spy` es un DataFrame válido
fig = df_spy.plot(x='Time', y=df_spy.columns)

fig.show()

### Obtención de datos adiccionales
Las medias móviles simples (SMA) y el Average True Range (ATR) son indicadores técnicos ampliamente utilizados en el análisis de series temporales financieras. 

1. La media móvil simple (SMA) es el promedio del precio de cierre de un activo en un número determinado de períodos. Su principal objetivo es suavizar los datos y ayudar a identificar la dirección de la tendencia del mercado. Se calcula sumando los precios de cierre de los últimos N períodos y dividiendo entre N.

## Obtener los 3 SMA

In [19]:
import requests
import pandas as pd

SYMBOL = "SPY"  # Activo a analizar
INTERVAL = "15min"

def get_sma(symbol, interval, time_period):
    """
    Obtiene la Media Móvil Simple (SMA) de un activo desde Alpha Vantage.
    
    Parámetros:
    - symbol: Ticker del activo (Ejemplo: "SPY")
    - time_period: Número de días para calcular la SMA (Ejemplo: 20, 50, 200)
    
    Retorna:
    - DataFrame con la SMA calculada
    """
    url = f"https://www.alphavantage.co/query?function=SMA&symbol={symbol}&interval={interval}&time_period={time_period}&series_type=close&apikey=GJT1CV11AHDS6WXS"
    response = requests.get(url)
    data = response.json()

    key_name = list(data.keys())[-1] if len(data.keys()) > 1 else None
    if not key_name or key_name == "Information":
        print(f"Error obteniendo SMA para {symbol} ({time_period} días).")
        print("Respuesta de la API:", data)
        return pd.DataFrame() 

    df = pd.DataFrame.from_dict(data[key_name], orient="index")
    df = df.rename(columns={"SMA": f"SMA_{time_period}"}) 
    df = df.astype(float)
    df.index = pd.to_datetime(df.index)
    return df.sort_index()

sma_20 = get_sma(SYMBOL, INTERVAL, 20)
sma_50 = get_sma(SYMBOL, INTERVAL, 50)
sma_200 = get_sma(SYMBOL, INTERVAL, 200)

# UNIR LAS TRES SMA EN UN SOLO DATAFRAME
df_sma = sma_20.join([sma_50, sma_200], how="outer")
df_sma


Error obteniendo SMA para SPY (20 días).
Respuesta de la API: {'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
Error obteniendo SMA para SPY (50 días).
Respuesta de la API: {'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
Error obteniendo SMA para SPY (200 días).
Respuesta de la API: {'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}


""


In [6]:
df_sma

,SMA_20,SMA_50,SMA_200
1999-11-29,88.5199,NaN,NaN
1999-11-30,88.6380,NaN,NaN
1999-12-01,88.8227,NaN,NaN
1999-12-02,89.0053,NaN,NaN
1999-12-03,89.2376,NaN,NaN
...,...,...,...
2025-02-21,604.9540,598.8254,566.1337
2025-02-24,604.4160,598.7161,566.5617
2025-02-25,604.1595,598.5849,566.9719
2025-02-26,603.6605,598.3667,567.3834


In [9]:
import requests
import pandas as pd

API_KEY = "TU_API_KEY"  
SYMBOL = "SPY"  
INTERVAL = "15min" 

# FUNCIÓN PARA OBTENER INDICADORES TÉCNICOS (MACD, ATR)
def get_technical_indicator(symbol, function, interval, params=""):
    """
    Obtiene un indicador técnico desde Alpha Vantage.

    Parámetros:
    - symbol: Ticker del activo (Ejemplo: "SPY")
    - function: Indicador a obtener (Ejemplo: "MACD", "ATR")
    - interval: Intervalo de tiempo (Ejemplo: "1min", "5min", "15min", "30min", "60min")
    - params: Parámetros adicionales para la API

    Retorna:
    - DataFrame con el indicador técnico calculado
    """
    url = f"https://www.alphavantage.co/query?function={function}&symbol={symbol}&interval={interval}{params}&apikey=YBKMA5L9K9I3GE0E"
    response = requests.get(url)
    data = response.json()

    key_name = list(data.keys())[-1] if len(data.keys()) > 1 else None
    if not key_name or key_name == "Information":
        print(f"Error obteniendo {function} para {symbol} (intervalo {interval}).")
        print("Respuesta de la API:", data)
        return pd.DataFrame()  # Devuelve un DataFrame vacío si hay error

    df = pd.DataFrame.from_dict(data[key_name], orient="index")
    df = df.astype(float)
    df.index = pd.to_datetime(df.index)  # Convertir índice a formato fecha-hora
    return df.sort_index()

atr = get_technical_indicator(SYMBOL, "ATR", INTERVAL, "&time_period=14")
atr = atr.rename(columns={"ATR": "ATR_15min"})





In [10]:
atr

,ATR_15min
2025-01-29 07:30:00,0.3828
2025-01-29 07:45:00,0.3804
2025-01-29 08:00:00,0.5190
2025-01-29 08:15:00,0.5362
2025-01-29 08:30:00,0.5400
...,...
2025-02-27 19:00:00,3.6058
2025-02-27 19:15:00,3.3946
2025-02-27 19:30:00,3.1965
2025-02-27 19:45:00,2.9960


In [ ]:
pd.options.plotting.backend = "plotly"

# Asegúrate de que `df_spy` es un DataFrame válido
fig = df_spy.plot(x='Time', y=df_spy.columns)

fig.show()

In [11]:
atr.reset_index(inplace=True)
atr.rename(columns={"index": "Time"}, inplace=True)
atr.head()

,Time,ATR_15min
0,2025-01-29 07:30:00,0.3828
1,2025-01-29 07:45:00,0.3804
2,2025-01-29 08:00:00,0.5190
3,2025-01-29 08:15:00,0.5362
4,2025-01-29 08:30:00,0.5400


In [13]:
pd.options.plotting.backend = "plotly"

# Asegúrate de que `df_spy` es un DataFrame válido
fig = atr.plot(x='Time', y=atr.columns)

fig.show()